## "A Bossy Sort of Voice"
#### A study of sexism in the Harry Potter series using Natural Language Processing

My eldest son is almost 6 and loves the Harry Potter series of books.  But, as I read them for the first time with him as a bedtime story, I noticed something hadn't expected.  

The sexism.

The sexism critique of the Harry Potter novels is not a new one - many people have (written)[https://www.bustle.com/articles/136244-the-5-least-feminist-moments-in-harry-potter] excellent articles about Ron's treatment of Hermoine, the portrayl of other female characters as cold or incompetent or promiscuous.  But there were two types of analyses I didn't see: a quantitative or linguistics based one, or something that looked at how the author herself portrays female characters in a biased light in how they speak

My hope in doing this is that people will use some of these tools to look more critically at the language of literature we love and the popular press for signs of gender, racial, and other bias.

#### Getting started: setting up hypotheses and requirements

In this project, I will test three hypotheses:

1. Female characters are referred to by the narrator with sexist words throughout the series by the narrator, while males are not described with that same language.

2. The narrator will use more sexist words when describing the female characters than other characters will use when talking about them.

3. Sexism as defined above will decline in both dialog and narration as the series progresses.


Taking this approach requires tools to do the following, which I'll tackle in this workbook:

1. Process text from all 7 'Harry Potter' books.

2. Seperate dialog from narration.

3. Label dialog and narration.

4. Distinguish parts of speech, like nouns, verbs, and adjectives.
000

2. Look at the text at the word and sentence level.

3. Distinguish parts of speech, like nouns, verbs, and adjectives.

4. 

5. Be able to tell female characters from male characters

6. Have examples of sexist words and/or phrases to create a classifier. 

7. Reduce sexist words in the text to their root forms (e.g. 'shriller' should be equivalent to 'shrill').

8. Summarize the data to test our hypotheses.


#### Step 1: Process text from all 7 'Harry Potter' books.

What I mean by "process", is to get the text from the series into a format that can be read into a computer program for analysis.  For this project, I'll be using the [Python programming language](https://www.python.org/), and a few libraries (basically groupings of code that completes specialized common processes), most notably the [Natural Language Processing Toolkit (NLTK)](http://www.nltk.org/).

To do this, I am going to read 7 files in the .txt format, each containing text of one of the books, using Python's built-in `open` function and `read` method.  You can get the files for these and other books from [this site](https://archive.org/stream/pdfy-ZhGUmtnn6LEtA7jL/Harry%20Potter%20and%20the%20Philosopher%27s%20Stone%2C%20by%20J.K.%20Rowling_djvu.txt) -- note that you can use existing txt format files, or copy text, paste it in Notepad and save as a .txt file.

In this code, the files are named 'hp' then the book number - i.e. `hp1.txt` - and stored in a folder called `corpus`. To run the code as it is, you'll need to recreate this schema or alter the code to fit the path you create.

Let's write a function to read in our text.  

In [1]:
def read_file(num):
    text = ''
    with open('corpus/hp'+ str(num) + '.txt', 'rt') as file_in:
        for line in file_in:
            text = text + line
    return text

book_content = read_file(1)

Running this would return a single long string of text, and it's now usable in our program.  Notice that we pass in the number of the book we want to open with the `num` variable.

#### Step 2: Tokenize the text
In the text above, the opening and closing quotes look the same: `"`.  However, when we tokenize the text, NLTK makes opening and closing quotes look different, so we can see where dialog begins and ends.  Let's give it a try!

In [2]:
from nltk import word_tokenize

def tokenize_text(book_content):
    tokenized = word_tokenize(book_content)
    return tokenized

tokenized = tokenize_text(book_content)

As you can see in the text above, our opening and closing quotes look like this: `'``' and "''"`.  This is a helpful tool in judging where dialog begins and ends across sentences.  Next, we will set up some rules around this and label parts of text.

#### Step 3: Label dialog and narration
In this step, we will label these two types of text while keeping them in order in case we need context.  To do this, we'll keep the list format to preserve order and create a tuple for each piece of dialogue.  So, for example:

`"''", 'That', 'can', 'be', 'rearranged', ',', "''", 'said', 'the', 'portrait', 'at', 'once', '.'`

Would become:

`('d', ["''", 'That', 'can', 'be', 'rearranged', ',', "''",]), ('n', ['said', 'the', 'portrait', 'at', 'once', '.'])`

What we'll do to achieve this is:
1. Create a new list called `parsed`.
1. Loop through the text in `tokenized` variable (printed above).  When we hit an open quote character, stop, grab everything up to that point and make it a list in a tuple where the first value is `n` for "narration", and the second value is a list containing all of those words (e.g. `('n', ['For', 'a', 'brief', 'moment', 'he', 'allowed', 'himself', 'the', 'impossible', 'hope', 'that', 'nobody', 'would', 'answer', 'him', '.', 'However', ',', 'a', 'voice', 'responded', 'at', 'once', ',', 'a', 'crisp', ',', 'decisive', 'voice', 'that', 'sounded', 'as', 'though', 'it', 'were', 'reading', 'a', 'prepared', 'statement', '.', 'It', 'was', 'coming', '--', 'as', 'the', 'Prime', 'Minister', 'had', 'known', 'at', 'the', 'first', 'cough', '--', 'from', 'the', 'froglike', 'little', 'man', 'wearing', 'a', 'long', 'silver', 'wig', 'who', 'was', 'depicted', 'in', 'a', 'small', ',', 'dirty', 'oil', 'painting', 'in', 'the', 'far', 'corner', 'of', 'the', 'room', '.',']`.  
2. Append this tuple to `text_parsed`.
3. Use the point where we found the open quote as a placeholder, then look ahead until we find a close quote.
4. Take the whole slice, from open quote to close quote.
4. Drop the slice into a tuple where the first value is `d` for "dialog" and the second value is a list containing all of the words and the quotes (e.g. `('d', '``', 'To', 'the', 'Prime', 'Minister', 'of', 'Muggles', '.', 'Urgent', 'we', 'meet', '.', 'Kindly', 'respond', 'immediately', '.', 'Sincerely', ',', 'Fudge', '.', "''")`)

The function to do this will be called `parse_text`. 

In [3]:
def parse_text(t):
    open_q = '``'
    close_q = "''"
    found_c = False # this will be used to break the while loop below
    # current will hold words until an open quote is found
    current = []
    
    parsed_dialog = [] 
    parsed_narrative = []
    length = len(t)
    i = 0

    while i < length:
        word = t[i]
        
        if word != open_q and word != close_q:
            current.append(word)

        elif word == open_q or word == close_q:
            parsed_narrative.append(current)
            
            current = []
            current.append(word)
            
            while found_c == False and i < length-1:
                i += 1
                if t[i] != close_q:
                    current.append(t[i])
                else:
                    current.append(t[i])
                    parsed_dialog.append(current)
                    current = []
                    found_c = True
        
        found_c = False
        i += 1
        
    return (parsed_dialog, parsed_narrative)
        

In [4]:
parsed_dialog, parsed_narrative = parse_text(tokenized)

# the text below is the same as above, now categorized with an 'n' (narration) or 'd' (dialog)
print("Sample of dialog list: ", parsed_dialog[:3])
print()
print("Sample of narrative list:", parsed_narrative[3:4])

Sample of dialog list:  [['``', 'Little', 'tyke', ',', "''"], ['``', 'The', 'Potters', ',', 'that', "'s", 'right', ',', 'that', "'s", 'what', 'I', 'heard', 'yes', ',', 'their', 'son', ',', 'Harry', "''"], ['``', 'Sorry', ',', "''"]]

Sample of narrative list: [['he', 'grunted', ',', 'as', 'the', 'tiny', 'old', 'man', 'stumbled', 'and', 'almost', 'fell', '.', 'It', 'was', 'a', 'few', 'seconds', 'before', 'Mr.', 'Dursley', 'realized', 'that', 'the', 'man', 'was', 'wearing', 'a', 'violet', 'cloak', '.', 'He', 'did', "n't", 'seem', 'at', 'all', 'upset', 'at', 'being', 'almost', 'knocked', 'to', 'the', 'ground', '.', 'On', 'the', 'contrary', ',', 'his', 'face', 'split', 'into', 'a', 'wide', 'smile', 'and', 'he', 'said', 'in', 'a', 'squeaky', 'voice', 'that', 'made', 'passersby', 'stare', ',']]



Now that we have our text organized the way we want it, we can run some analysis on the narration to see how Rowling portrays female characters.   There are a couple of challenges with this.

First, we need to isolate narrative passages that refer to female characters, either by name or pronoun.

The next challenge is determining how do we know something is sexist?  There are a couple of ways to approach this.  The first is to determine what rules we will use to determine something is sexist, then apply those rules to the passages we isolate.  The other would be to create an algorithm, feed it a labeled training set of sexist phrases and train it to recognize others.

For the purposes of this project, I'm going to use the first approach: writing a set of rules and testing each narrative passage against it.  This is more transparent and in my opinion less subject to bias than finding and labeling passages of text, and possibly less error prone.


#### Step 4: Isolating the right narrative passages
Let's create a list of all of the narrative that describes a female character.  The most basic step is creating a list of all of the narrative passages.  

To do this, we're going to loop through our full text variable, `tagged`, and pull out only the parts marked with `n` in the `isolate narrative` function below.

In [33]:
from itertools import groupby as gb

p_list = ['Harry', 'Ron', 'Hermione']

def split_sent(t):
    sent_list = []
    for sent in t:
        k = [list(sent) for i, sent in gb(sent, lambda item: item=='.')]
        for i in k:
            if len(i) > 1:
                sent_list.append(i)
    return (sent_list)

def protagonist(n, p_list):
    protagonist_narrative = {
        'Harry': [],
        'Ron':  [],
        'Hermione':  [],
    }
    for i in n:
        for word in i:
            
            if word in p_list:
                if word == p_list[0]:
                    protagonist_narrative['Harry'].append(i)
                if word == p_list[1] :
                    protagonist_narrative['Ron'].append(i)
                if word == p_list[2]:
                    protagonist_narrative['Hermione'].append(i)
                    
    return protagonist_narrative
        

narrative_split_sent = split_sent(parsed_narrative)
protagonist_dict = protagonist(narrative_split_sent, p_list)

In [6]:
# Let's print the first few entries from the  key to see 
print("*** Sample from the 'Harry' key ***")
print(protagonist_dict['Harry'][:5])
print("*** Sample from the 'Ron' key ***")
print(protagonist_dict['Ron'][:5])
print("*** Sample from the 'Hermione' key ***")
print(protagonist_dict['Hermione'][:5])

*** Sample from the 'Harry' key ***
[['He', 'was', 'sure', 'there', 'were', 'lots', 'of', 'people', 'called', 'Potter', 'who', 'had', 'a', 'son', 'called', 'Harry'], ['Come', 'to', 'think', 'of', 'it', ',', 'he', 'was', "n't", 'even', 'sure', 'his', 'nephew', 'was', 'called', 'Harry'], ['She', 'eyed', 'his', 'cloak', 'suddenly', 'as', 'though', 'she', 'thought', 'he', 'might', 'be', 'hiding', 'Harry', 'underneath', 'it'], ['Dumbledore', 'took', 'Harry', 'in', 'his', 'arms', 'and', 'turned', 'toward', 'the', 'Dursleys', "'", 'house'], ['He', 'bent', 'his', 'great', ',', 'shaggy', 'head', 'over', 'Harry', 'and', 'gave', 'him', 'what', 'must', 'have', 'been', 'a', 'very', 'scratchy', ',', 'whiskery', 'kiss']]
*** Sample from the 'Ron' key ***
[['said', 'Ron'], ['said', 'Ron', 'again'], ['mumbled', 'Ron'], ['said', 'Harry', 'and', 'Ron'], ['Ron', 'blurted', 'out']]
*** Sample from the 'Hermione' key ***
[['said', 'Hermione'], ['said', 'Hermione'], ['He', 'was', 'just', 'taking', 'Harry', '

##### Step 5: Distinguish parts of speech, like nouns, verbs, and adjectives
Now that we've categorized text as narration or dialog, we can use NLTK's classification function to categorize words by the part of speech they represent.

The `pos_tag` function of NLTK that we'll use to do this takes in a word as a string and returns a tuple of the word and a code for how it was classified.  For example, 'NNP' means the word has been tagged by NLTK as a proper noun, 'VB' means the word has been tagged as a verb.  So "Harry" in the sentence "Harry is Petunia's nephew." would come back as `('Harry', 'NNP')`.  

You can find a full list [here](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/), but in the next steps we'll be focused primarily on adjectives, verbs and nouns.

We will import the `pos_tag` function from NLTK and tag the words in a new function called `tagged_text`.

In [7]:
from nltk import pos_tag

def tagged_text(i):
    tagged = [pos_tag(word) for word in i]
    return tagged


In [8]:
# Let's print the output of tagged text for the first few sentences that mention 'Harry' to see what this looks like
print(tagged_text(protagonist_dict['Harry'][:50]))

[[('He', 'PRP'), ('was', 'VBD'), ('sure', 'RB'), ('there', 'EX'), ('were', 'VBD'), ('lots', 'NNS'), ('of', 'IN'), ('people', 'NNS'), ('called', 'VBN'), ('Potter', 'NNP'), ('who', 'WP'), ('had', 'VBD'), ('a', 'DT'), ('son', 'NN'), ('called', 'VBN'), ('Harry', 'NNP')], [('Come', 'NNP'), ('to', 'TO'), ('think', 'VB'), ('of', 'IN'), ('it', 'PRP'), (',', ','), ('he', 'PRP'), ('was', 'VBD'), ("n't", 'RB'), ('even', 'RB'), ('sure', 'JJ'), ('his', 'PRP$'), ('nephew', 'NN'), ('was', 'VBD'), ('called', 'VBN'), ('Harry', 'NNP')], [('She', 'PRP'), ('eyed', 'VBD'), ('his', 'PRP$'), ('cloak', 'NN'), ('suddenly', 'RB'), ('as', 'IN'), ('though', 'IN'), ('she', 'PRP'), ('thought', 'VBD'), ('he', 'PRP'), ('might', 'MD'), ('be', 'VB'), ('hiding', 'VBG'), ('Harry', 'NNP'), ('underneath', 'IN'), ('it', 'PRP')], [('Dumbledore', 'NNP'), ('took', 'VBD'), ('Harry', 'NNP'), ('in', 'IN'), ('his', 'PRP$'), ('arms', 'NNS'), ('and', 'CC'), ('turned', 'VBD'), ('toward', 'IN'), ('the', 'DT'), ('Dursleys', 'NNP'), ("'

In [30]:
from collections import Counter
# **PROBLEM AREA 1**
def parse_tagged(protagonist_dict):
    tagged_dict = {}
    descriptive_verbs = {}
    descriptive_adj = {}
    
    for k, v in protagonist_dict.items():
        # note that the dictionary is created while calling the tagged_text helper function
        tagged_dict[k] = tagged_text(protagonist_dict[k])
        # create a list under the key for the protagonist's name 
        descriptive_verbs[k] = []
        descriptive_adj[k] = []
        
        # iterate through the 
        for sent in tagged_dict[k]:
            for n in range(len(sent)):
                # check if the tuple is the protagonist's name
                if sent[n] == (k, 'NNP') or sent[n] == (k, 'NN'): 
                    # this case is for "noun verb" e.g. "harry said"
                    try:
                        # this case is for "noun verb" e.g. "harry said"
                        if 'VB' or 'RB' in sent[n+1][1]:
                            descriptive_verbs[k].append(sent[n+1][0])
                        
                        try:
                            # this case is for "noun verb adverb" e.g. "harry said angrily"
                            if 'VB' or 'RB' in sent[n+2][1]:
                                descriptive_verbs[k].append(sent[n+2][0])
                            # this case is for "noun verb adjective" e.g. "harry is thin"
                            elif 'JJ' or 'ADJ' in sent[n+2][1]:
                                descriptive_adj[k].append(sent[n+2][0])
                        except:
                            continue

                    
                    except:
                        continue
                        
                    try:
                        # eg. "muttered harry"
                        if 'VB' or 'RB' in sent[n-1][1]:
                            descriptive_verbs[k].append(sent[n+1][0])
                        # eg. "handsome harry"
                        elif 'JJ' or 'ADJ' in sent [n-1][1]:
                            descriptive_adj[k].append(sent[n-1][0])
                    except:
                        continue
                    
                    
                    
                        

    return (descriptive_verbs, descriptive_adj, tagged_dict)
            
descriptive_verbs, descriptive_adj, tagged_dict = parse_tagged(protagonist_dict)
print(descriptive_adj)

{'Harry': [], 'Ron': [], 'Hermione': []}


In [ ]:
hp = descriptive_verbs['Harry']
hg = descriptive_verbs['Hermione']
rw = descriptive_verbs['Ron']
print(rw)

In [ ]:
def hermione_only(descriptive):
    hg_set, hp_set, rw_set = set(descriptive['Hermione']), set(descriptive['Harry']), set(descriptive['Ron'])
    for word in hg_set:
        if word not in hp_set and word not in rw_set:
            print(word)
            
print(descriptive_verbs['Harry'])

In [ ]:
# this function 
def descriptor_verbs_adverbs(pd):
    speech_descriptors = {}
    hermione = []
    harry = []
    ron = []
    
    for k, v in pd.items():
        cnt = Counter()
        
        for value in v:
            s = pos_tag(value)

            for i in range(len(s)):
                try:
                    # cases of VERB-NOUN-ADVERB e.g. "said Ron angrily"
                    if 'VB' in s[i][1] and s[i+1][0] == k and s[i+1][1]:
                        if s[i+2][1] == 'RB':
                            cnt[s[i][0]] += 1
                            cnt[s[i+2][0]] += 1
                            if k == 'Hermione':
                                hermione.append(s[i+2][0])
                                hermione.append(s[i][0])
                            if k == 'Harry':
                                harry.append(s[i+2][0])
                                harry.append(s[i][0])
                            if k == 'Ron':
                                ron.append(s[i][0])
                                ron.append(s[i+2][0])
                except:
                    pass
                
                try:
                    # cases of NOUN-VERB-ADVERB e.g. "Ron said angrily"
                    if s[i][0] == k and 'VB' in s[i+1][1]:
                        cnt[s[i+1][0]] += 1
                        try:
                            if s[i+2][1] =='RB':
                                cnt[s[i+1][0]] += 1
                                cnt[s[i+2][0]] += 1
                                if k == 'Hermione':
                                    hermione.append(s[i+1][0])
                                    hermione.append(s[i+2][0])
                                if k == 'Harry':
                                    harry.append(s[i+1][0])
                                    harry.append(s[i+2][0])
                                if r == 'Ron':
                                    ron.append(s[i+1][0])
                                    ron.append(s[i+2][0])
                        except:
                            pass
                except:
                    pass
        
        speech_descriptors[k] = cnt
        
    return (speech_descriptors, hermione, harry, ron)

speech, hermione, harry, ron = descriptor_verbs_adverbs(protagonist_dict)


In [ ]:
import matplotlib.pyplot as plt
# from PIL import Image
import wordcloud
from wordcloud import WordCloud
#generate the word cloud with parameters

wc = WordCloud(background_color="white", 
               max_words=75, 
               min_font_size =5, 
               max_font_size=40, 
               relative_scaling = 0.4, 
               normalize_plurals= True)

fig_sz = (20,20)

#harry
print ("*" * 20, "Harry", "*" * 20)
wc.generate(' '.join(harry))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

plt.show()

#ron
print ("*" * 20, "Ron", "*" * 20)
wc.generate(' '.join(ron))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

#Show the wordcloud
plt.show()

#hermione
print ("*" * 20, "Hermione", "*" * 20)
wc.generate(' '.join(hermione))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

#Show the wordcloud
plt.show()

In [ ]:
def character_exclusive(char, speech, p_list):
    char_list = speech[char].keys()
    other_chars = []
    
    for item in p_list:
        if item != char and item != "Ronald":
            other_chars.extend(speech[item])
    
    common_words = set(other_chars)
    exclusive_words = []
    
    for w in char_list:
        if w not in common_words:
            for n in range(speech[char][w]):
                exclusive_words.append(w)
    
    return exclusive_words
    
    
hermione_excl = character_exclusive('Hermione', speech, p_list)
harry_excl = character_exclusive('Harry', speech, p_list)
ron_excl = character_exclusive('Ron', speech, p_list)

print(Counter(harry_excl).most_common(10))
print(Counter(ron_excl).most_common(10))
print(Counter(hermione_excl).most_common(10))

In [ ]:
#harry only
print ("*" * 20, "Harry", "*" * 20)
wc.generate(' '.join(harry_excl))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

plt.show()

#ron only
print ("*" * 20, "Ron", "*" * 20)
wc.generate(' '.join(ron_excl))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

#Show the wordcloud
plt.show()

#hermione
print ("*" * 20, "Hermione", "*" * 20)
wc.generate(' '.join(hermione_excl))
plt.figure(figsize=fig_sz)
plt.imshow(wc)
plt.axis("off")

#Show the wordcloud
plt.show()

Finding adjectives is a little more difficult, as they can be disbursed in the sentence.  But, there are rules that apply to adjectives that we can leverage.

First off, adjectives tend to appear in sentences in this order:
1. Quantity or number
2. Quality or opinion
3. Size
4. Age
5. Shape
6. Color
7. Proper adjective (often nationality, other place of origin, or material)
8. Purpose or qualifier

If two or more adjectives are from the same group in the list above, the word "and" is placed between the two adjectives.  (e.g. "The hall was decorated in red and green streamers.")

If there are three or more adjectives from the same group, there will be a comma between each of the adjectives. (e.g. "The hall was decorated in red, white and green streamers." 

What does this mean for us 

In [36]:
def find_adjectives(tagged_dict, p_list):
    # create a dictionary to store adjectives
    descriptive_adj = {}
    for p in p_list:
        descriptive_adj[p] = []
    
    # loop through the tagged narrative pieces
    for k, v in tagged_dict.items():
        for sent in v:
            # this variable will be our switch to turn adjective collection on or off
            append_jj = False
            for word in sent:
                # if the word is a noun and it's referring to the character whose adjectives we're collecting
                if 'NN' in word[1] and word[0] == k:
                    append_jj = True
    
    

    
    
                
                
                    

find_adjectives(tagged_dict, p_list)
    

[('He', 'PRP'), ('was', 'VBD'), ('sure', 'RB'), ('there', 'EX'), ('were', 'VBD'), ('lots', 'NNS'), ('of', 'IN'), ('people', 'NNS'), ('called', 'VBN'), ('Potter', 'NNP'), ('who', 'WP'), ('had', 'VBD'), ('a', 'DT'), ('son', 'NN'), ('called', 'VBN'), ('Harry', 'NNP')]
[('Come', 'NNP'), ('to', 'TO'), ('think', 'VB'), ('of', 'IN'), ('it', 'PRP'), (',', ','), ('he', 'PRP'), ('was', 'VBD'), ("n't", 'RB'), ('even', 'RB'), ('sure', 'JJ'), ('his', 'PRP$'), ('nephew', 'NN'), ('was', 'VBD'), ('called', 'VBN'), ('Harry', 'NNP')]
[('She', 'PRP'), ('eyed', 'VBD'), ('his', 'PRP$'), ('cloak', 'NN'), ('suddenly', 'RB'), ('as', 'IN'), ('though', 'IN'), ('she', 'PRP'), ('thought', 'VBD'), ('he', 'PRP'), ('might', 'MD'), ('be', 'VB'), ('hiding', 'VBG'), ('Harry', 'NNP'), ('underneath', 'IN'), ('it', 'PRP')]
[('Dumbledore', 'NNP'), ('took', 'VBD'), ('Harry', 'NNP'), ('in', 'IN'), ('his', 'PRP$'), ('arms', 'NNS'), ('and', 'CC'), ('turned', 'VBD'), ('toward', 'IN'), ('the', 'DT'), ('Dursleys', 'NNP'), ("'", '

In [ ]:
def descriptor_adjectives(pd):
    speech_descriptors = {}
    hermione = []
    harry = []
    ron = []
    
     for k, v in pd.items():
        cnt = Counter()
    
    for k, v in pd.items():
        cnt = Counter()
        
        for value in v:
            s = pos_tag(value)

            for i in range(len(s)):
                try:
                    # cases of ADJECTIVE-NOUN e.g. "disgruntled Hermione"
                    if 'JJ' in s[i][1] and s[i+1][0] == k:
                            cnt[s[i][0]] += 1
                            if k == 'Hermione':
                                hermione.append(s[i][0])
                            if k == 'Harry':
                                harry.append(s[i][0])
                            if k == 'Ron':
                                ron.append(s[i][0])
                                
                except:
                    pass
                
                # cases where the protagonist name 
                
                try:
                    # cases of NOUN-VERB-ADJECTIVE e.g. "Hermione seemed disgruntled"
                    if s[i][0] == k and 'VB' in s[i+1][1]:
                        try:
                            if s[i+2][1] =='JJ':
                                cnt[s[i+2][0]] += 1
                                if k == 'Hermione':
                                    hermione.append(s[i+2][0])
                                    print(k, s[i+2], s)
                                if k == 'Harry':
                                    harry.append(s[i+2][0])
                                if r == 'Ron':
                                    ron.append(s[i+2][0])
                        except:
                            pass
                except:
                    pass
        
        speech_descriptors[k] = cnt
        
    return (speech_descriptors, hermione, harry, ron)

speech, hermione1, harry1, ron1 = descriptor_adjectives(protagonist_dict)
print(hermione1, harry1, ron1)

In [ ]:
from nltk.collocations import BigramAssocMeasures, TrigramAssocMeasures

def colloc(isolate_narrative):
    # first we need to mash the narrative pieces together and remove the 'n' at the start of each tuple
    mashed_narrative = [n[1] for n in isolate_narrative ]
    mn = [i for mn in mashed_narrative for i in mn]

    bam = BigramAssocMeasures()
    tam = TrigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(mn)
    
    like = sorted(finder.nbest(tam.raw_freq, 200))
    print(like)
colloc(isolate_narrative)

#### Step 5: Isolating sexist language
This part is tricky - sexism can be subtle or detectable only with context.  To keep this analysis straightforward, I assembled a list of sexist works to search for in the narrative.

Because the Harry Potter series is written in English by a British writer, I focused on sources from the UK and countries in the Commonwealth.  Using this blog post by a [New Zealand blogger](http://sacraparental.com/2016/05/14/everyday-misogyny-122-subtly-sexist-words-women/) I had a first set of words and some excellent categories to begin with.  I found a number of [other](http://time.com/4268325/history-calling-women-shrill/) excellent articles about sexism in language, which I used to add to the `sexist_words` Python dictionary below, grouping them by type.

In [ ]:
sexist_words = { 
    'assertiveness': ['bossy', 'abrasive', 'ball-bust', 'aggressive', 'shrill', 'bolshy', 'intense', 'stroppy', 'mannish', 'strident', 'know-it-all'],
    'behavior' : ['cackle', 'shriek', 'giggl', 'caterwaul', 'yowl', 'screech','gossip', 'dramatic', 'catty', 'bitch', 'nag', 'coldly', 'icy', 'shrew', 'humorless', 'man-hater', 'banshee', 'fishwife', 'lippy', 'ditzy', 'diva', 'prima donna', 'feisty', 'ladylike', 'bubbly', 'vivaious', 'flirt', 'sass', 'chatty', 'demure', 'modest', 'emotional', 'hysterical', 'hormonal', 'menstrual', 'flaky', 'over-sensitive'],
    'sexuality': ['slut', 'trollop', 'frigid', 'tease', 'loose', 'man-eater', 'prude', 'curvy', 'cheap', 'frump', 'mouse', 'clotheshorse', 'hag'],
    'relationship': ['spinster', 'barren', 'housewife', 'houseproud', 'mistress'],
#     'praise': ['care', 'compassion', 'hard-working', 'conscientious', 'dependable', 'diligent', 'dedicated', 'tactful', 'interpersonal', 'warm', 'helpful'],
}

# making this into a list for easier analysis
sexist_words_list = [v[j] for k, v in sexist_words.items() for j in range(len(v))]

Now that we know what words we are looking for, we want to make sure we get all versions of the words; if we only compare them to this list, we'd pick up on `cackle` but not `cackled`.   

To to this, we'll use a stemming algorithm that's part of NLTK.  Let's look at some examples of how this works.  What we want to happen is to have all forms of a word have the same stem.

In [ ]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

# this works really well for some words
print("Bossiness stem:", st.stem('bossiness'))
print("Bossily stem:", st.stem('bossily'))
print("Bossy stem", st.stem('bossy'))

print("Cackled stem:", st.stem('cackled'))
print("Cackling stem:", st.stem('cackling'))
print("Cackle stem:", st.stem('cackle'))

# but not for others
print("Shreiking stem:", st.stem('shrieking'))
print("Shrieked stem:", st.stem('shrieked'))
print("Shriek stem:", st.stem('shriek'))


As we can see above, this doesn't *always* work, so for extra insurance, we'll have a rule that if the root word is in a word, it counts. 

So, even though the stemmer isn't linking `shrieking` to `shreik`, looking for the letters in `shreik` would.

In [ ]:
def sexist_words(t):
    sexist_words_isolated = []
    sexist_words_count = Counter()
    for s in t: # (n, [()])
        sent = pos_tag(s)
        nouns = [j[0] for j in sent if 'NN' in j[1]]
        for word in sent:
            if st.stem(word[0]).lower() in sexist_words_list and word != "Moody":
                sexist_words_isolated.append(word[0].lower())
                sexist_words_count[word[0].lower()] +=1
    return (sexist_words_isolated, sexist_words_count)

p, count = sexist_words(narrative_split_sent)

#sexist words
print ("*" * 20, "Sexist words", "*" * 20)
wc.generate(' '.join(p))
plt.figure(figsize=(25,25))
plt.imshow(wc)
plt.axis("off")

#Show the wordcloud
plt.show()

print(count)

In [ ]:
1: 'icy': 4, 'screech': 3, 'bossy': 2, 'nagging': 2, 'shriek': 2, 'gossiped': 1, 'shrill': 1, 'cheap': 1, 'hags': 1, 'know-it-all': 1}
2: Counter({'screech': 3, 'shrill': 3, 'hags': 2, 'icy': 2, 'haggis': 2, 'bossily': 1, 'icily': 1, 'shriek': 1})